In [1]:
#IMPORTS

import matplotlib
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy
import scipy.interpolate
import mne
import scipy
import statistics
from pywt import wavedec
from scipy import signal
import matplotlib.colors as mcolors
import random
import seaborn as sns
import os

In [2]:
from Definicions import upperchanel
from Definicions import group_inf
from Definicions import mediumchanels
from Definicions import opteciogrups
from Definicions import find_nearest
from Definicions import grabt
from Definicions import bandpower
from Definicions import epoch_return

In [3]:
#Init. info.
#GROUPS
group1= ["FP2", "AFZ", "FZ", "FCZ", "CZ"]
group2= ["F7", "F5", "F3", "F1", "AF7", "AF3", "FP1"]
group3= ["FP2", "AF4", "AF8", "F2", "F4", "F6", "F8"]
group4= ["FT7", "T7", "TP7"]
group5= ["FC5", "FC3", "FC1", "C5", "C3", "C1"]
group6= ["CP5", "CP3", "CP1", "P7", "P5", "P3", "P1"]
group7= ["PO7", "PO3", "O1", "PO9"]
group8= ["PO4", "PO8", "O2", "PO10"]
group9= ["P2", "P4", "P6", "P8", "CP2", "CP4", "CP6"]
group10= ["FT8", "T8", "TP8"]
group11= ["CPZ", "PZ", "POZ", "OZ", "IZ"]
group12= ["FC2", "FC4", "FC6", "C2", "C4", "C6"]

#X
groups = [group1, group2, group3, group4, group5, group6, group7, group8, group9, group10, group11, group12]

#General aspects
titlegrup = ["ch1","ch2","ch3","ch4","ch5","ch6","ch7","ch8","ch9","ch10","ch11","ch12"]
ct_ty = ["eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg"]
sfreq = 250
info = mne.create_info(ch_names=titlegrup, sfreq=sfreq, ch_types=ct_ty)

In [4]:
#Load
basePATH = 'Files/Preprocessed/'
listPATHEC = []
listPATHEO = []

for root, dirs, files in os.walk("Files/Preprocessed/"):
    for filename in files:
        if '.set' in filename:
            if 'EC' in filename:
                listPATHEC.append(filename)
            if 'EO' in filename:
                listPATHEO.append(filename)

sorted_list_EO = sorted(listPATHEO)
sorted_list_EC = sorted(listPATHEC)


In [5]:
EO_EC_Pacients = []

for i in sorted_list_EO:
    #LOAD
    x=mne.io.read_raw_eeglab(basePATH+i, preload=True, verbose=True)
    #GET DATA
    data = x._data
    #GET CHANELS
    chanles_names = x.ch_names
    #REDUCE CHANELS TO 12
    EO = opteciogrups(chanles_names, groups, data)

    #Select the same but with EC
    EC_name = [sorted_list_EC.index(e) for e in sorted_list_EC if i[:-6] in e]
    EC_name_PATH = sorted_list_EC[EC_name[0]]
    #LOAD
    x2=mne.io.read_raw_eeglab(basePATH+EC_name_PATH, preload=True, verbose=True)
    #GET DATA
    data2 = x2._data
    #GET CHANELS
    chanles_names2 = x2.ch_names
    #REDUCE CHANELS TO 12
    EC = opteciogrups(chanles_names2, groups, data2)

    #save
    EO_EC_P = []
    EO_EC_P.append(EO)
    EO_EC_P.append(EC)
    EO_EC_Pacients.append(EO_EC_P)

Reading Files/Preprocessed/sub-010002_EO.fdt
Reading 0 ... 117860  =      0.000 ...   471.440 secs...
Reading Files/Preprocessed/sub-010002_EC.fdt
Reading 0 ... 119344  =      0.000 ...   477.376 secs...
Reading Files/Preprocessed/sub-010003_EO.fdt
Reading 0 ... 119999  =      0.000 ...   479.996 secs...
Reading Files/Preprocessed/sub-010003_EC.fdt
Reading 0 ... 119515  =      0.000 ...   478.060 secs...
Reading Files/Preprocessed/sub-010004_EO.fdt
Reading 0 ... 119748  =      0.000 ...   478.992 secs...
Reading Files/Preprocessed/sub-010004_EC.fdt
Reading 0 ... 119277  =      0.000 ...   477.108 secs...
Reading Files/Preprocessed/sub-010005_EO.fdt
Reading 0 ... 119821  =      0.000 ...   479.284 secs...
Reading Files/Preprocessed/sub-010005_EC.fdt
Reading 0 ... 119999  =      0.000 ...   479.996 secs...
Reading Files/Preprocessed/sub-010006_EO.fdt
Reading 0 ... 110270  =      0.000 ...   441.080 secs...
Reading Files/Preprocessed/sub-010006_EC.fdt
Reading 0 ... 116011  =      0.000 ..

In [54]:
'''for pacient in EO_EC_Pacients:
    for data in pacient:
        matplotlib.pyplot.psd(data[1])
        plt.show()'''

'for pacient in EO_EC_Pacients:\n    for data in pacient:\n        matplotlib.pyplot.psd(data[1])\n        plt.show()'

In [10]:
#Subject = EO_EC_Pacients[S] max=15
#EO = EO_EC_Pacients[S][0]
#EC = EO_EC_Pacients[S][1]
#Channel = EO_EC_Pacients[0][1][Num] max=12
pacient_beta = []

for pacient in EO_EC_Pacients:
    EO_Epochs = epoch_return(pacient[0])
    chanels_betta = []
    for chane in EO_Epochs:
        for epoch in chane:
            f_beta = bandpower(epoch, 250, [13, 32],relative=True)
            chanels_betta.append(f_beta)
            #gamma
            f_gama = bandpower(epoch, 250, [32, 100],relative=True)
            #alpha
            f_alpha = bandpower(epoch, 250, [9, 13],relative=True)
            #theta
            f_theta = bandpower(epoch, 250, [4, 8],relative=True)
            #delta
            f_delta = bandpower(epoch, 250, [0.1, 4],relative=True)

    pacient_beta.append(chanels_betta)
    EC_Epochs = epoch_return(pacient[1])
    for chane in EC_Epochs:
        for epoch in chane:
            f_beta = bandpower(epoch, 250, [13, 32],relative=True)
            #gamma
            f_gama = bandpower(epoch, 250, [32, 100],relative=True)
            #alpha
            f_alpha = bandpower(epoch, 250, [9, 13],relative=True)
            #theta
            f_theta = bandpower(epoch, 250, [4, 8],relative=True)
            #delta
            f_delta = bandpower(epoch, 250, [0.1, 4],relative=True)


In [19]:
# compare beg. scale.
file_name =  'Files/TEIQue-SF.xlsx'

import pandas as pd
df_emotionality = pd.read_excel(io=file_name, usecols="A,D")
df_welbeing = pd.read_excel(io=file_name, usecols="A,F")

In [53]:
for i in range (0,len(sorted_list_EO)):
    nom = sorted_list_EO[i]
    rname = nom[:-7]
    class_23 = df_emotionality[df_emotionality["ID"].isin([str(rname)])]
    print(class_23)


            ID  TeiQueSF_emotionality
38  sub-010002                 5875.0
           ID  TeiQueSF_emotionality
3  sub-010003                    5.0
            ID  TeiQueSF_emotionality
39  sub-010004                 5625.0
             ID  TeiQueSF_emotionality
215  sub-010005                 6625.0
            ID  TeiQueSF_emotionality
33  sub-010006                   5.25
            ID  TeiQueSF_emotionality
46  sub-010007                    4.0
           ID  TeiQueSF_emotionality
8  sub-010010                 3375.0
            ID  TeiQueSF_emotionality
34  sub-010012                 4875.0
           ID  TeiQueSF_emotionality
9  sub-010016                 6125.0
           ID  TeiQueSF_emotionality
0  sub-010017                 4875.0
            ID  TeiQueSF_emotionality
10  sub-010019                 5875.0
            ID  TeiQueSF_emotionality
23  sub-010020                 4875.0
           ID  TeiQueSF_emotionality
5  sub-010021                    5.0
           ID  TeiQu

In [40]:
class_23 = df_emotionality[df_emotionality["ID"].isin(['sub-010007'])]
class_23
s = class_23['TeiQueSF_emotionality']
s

46    4.0
Name: TeiQueSF_emotionality, dtype: float64